### Training and testing of the GPT-2 model on the train dataset (Reddit book reviews)

In [31]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer
from datasets import load_dataset
import torch
from transformers import AutoModelForSequenceClassification
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
 

In [32]:
train_dataset = load_dataset("csv", data_files="books_train.csv")["train"]
test_dataset = load_dataset("csv", data_files="books_test.csv")["train"]

In [33]:
device = torch.device('cuda')

In [34]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [35]:
def preprocess_function(examples):
    inputs = tokenizer(examples["body"], truncation=True, return_tensors="pt", padding=True).to(device)
    return inputs
 

In [36]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [37]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [38]:
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2).to(device)
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
def compute_metrics(eval_pred):
   load_accuracy = evaluate.load("accuracy")
   load_f1 = evaluate.load("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)
   f1 = load_f1.compute(predictions=predictions, references=labels)
   return {"accuracy": accuracy, "f1": f1}

In [40]:
from huggingface_hub import notebook_login
notebook_login()

In [41]:
repo_name = "finetuning-gpt2-Reddit-books"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
   use_cpu=False
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


/tmp/ipykernel_3532/2786923507.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [42]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=400, training_loss=1.9761312866210938, metrics={'train_runtime': 120.0228, 'train_samples_per_second': 3.333, 'train_steps_per_second': 3.333, 'total_flos': 209037400473600.0, 'train_loss': 1.9761312866210938, 'epoch': 2.0})

In [43]:
trainer.evaluate()

{'eval_loss': 2.2443878650665283,
 'eval_accuracy': {'accuracy': 0.71},
 'eval_f1': {'f1': 0.8304093567251462},
 'eval_runtime': 8.044,
 'eval_samples_per_second': 12.432,
 'eval_steps_per_second': 12.432,
 'epoch': 2.0}

## The accuracy is 0.71, while the F1-score is 0.8304093567251462